In [1]:
# !pip install pybullet
# !pip install einops
# !pip install tqdm

# !conda install -c conda-forge ipywidgets
# !pip install jupyter_contrib_nbextensions
# !jupyter nbextension enable --py widgetsnbextension

In [1]:
import importlib.util
import pybullet_data
import scipy
import sys
import pybullet as pb
import numpy as np

from fairmotion.ops import conversions
from fairmotion.core.motion import Motion

import bullet_client
from data_utils import draw_ori
from preprocess_DIP_TC_new import load_and_augment_dip_motion
from bullet_agent import SimAgent
import constants as cst

[SimAgent] Creating an agent... data/amass.urdf
data/source/TotalCapture\s1\acting1_poses.npz data/source/TotalCapture_60FPS_Original/TotalCapture\s1_acting1.pkl
data/preprocessed_TotalCapture_v1/tcimu_TotalCapture\s1_acting1.pkl
already generated
data/source/TotalCapture\s1\acting2_poses.npz data/source/TotalCapture_60FPS_Original/TotalCapture\s1_acting2.pkl
data/preprocessed_TotalCapture_v1/tcimu_TotalCapture\s1_acting2.pkl
already generated
data/source/TotalCapture\s1\acting3_poses.npz data/source/TotalCapture_60FPS_Original/TotalCapture\s1_acting3.pkl
data/preprocessed_TotalCapture_v1/tcimu_TotalCapture\s1_acting3.pkl
already generated
data/source/TotalCapture\s1\freestyle1_poses.npz data/source/TotalCapture_60FPS_Original/TotalCapture\s1_freestyle1.pkl
data/preprocessed_TotalCapture_v1/tcimu_TotalCapture\s1_freestyle1.pkl
already generated
data/source/TotalCapture\s1\freestyle2_poses.npz data/source/TotalCapture_60FPS_Original/TotalCapture\s1_freestyle2.pkl
data/preprocessed_Total

In [10]:
DT = 1. / 60
acc_fd_N = 4
DT_FIN_ACC = DT * acc_fd_N
WIN_LEN = 11
NOMINAL_H = 1.7
rot_up_R = cst.rot_up_R

np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(suppress=True)

motion_name_imu = 'data/source/TotalCapture_60FPS_Original/TotalCapture/s1_freestyle3.pkl'
motion_name_gt = 'data/source/TotalCapture/s1/freestyle3_poses.npz'
# motion_name_gt = motion_name_imu = 'data/source/DIP_IMU/s_01/03.pkl'
# motion_name_gt = motion_name_imu = 'data/source/DIP_IMU/s_10/05.pkl'

ROOT_COM_OFFSET = np.array([0.0, 0.1, -0.1])        # 0.1m up, -0.1m back
# 1 TC, 0 DIP
if 1:
    ROT_MAT = conversions.A2R(np.array([np.pi / 2, 0, 0]))
else:
    ROT_MAT = rot_up_R

In [11]:
import pickle

In [14]:
def load(name):
    if name.endswith("npz"):
        data = np.load(name)
    elif name.endswith("pkl"):
        with open(name, "rb") as f:
            data = pickle.load(f, encoding="latin1")
    else:
        assert False

    return data

In [17]:
data_imu = load(motion_name_imu)

In [20]:
data_imu.keys()
data_imu["acc"].shape, data_imu["gt"].shape, data_imu["ori"].shape

((2039, 6, 3), (2039, 72), (2039, 6, 3, 3))